In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd

In [4]:
input_ranking=[x for x in range(10000)]
protected_group=[x for x in range(5000)]
unfair_ranking=dataGenerator.generateUnfairRanking(input_ranking,protected_group,0.2)


In [5]:
def hest(x):
    if x>=5000: 
        return 0
    else:
        return 1

In [6]:
unfair_rankingdf = pd.DataFrame(unfair_ranking, columns=["item"]) 
unfair_rankingdf
unfair_rankingdf["user"] = 1
unfair_rankingdf["Algorithm"] = 1
unfair_rankingdf["protected"] = unfair_rankingdf["item"].apply(hest)
unfair_rankingdf.index.names = ['rank']
unfair_rankingdf

#genre_stats['total_count_ratings'] = genre_stats['genres'].apply(app_total_count_ratings)

,item,user,Algorithm,protected
rank,,,,
0,5000,1,1,0
1,0,1,1,1
2,1,1,1,1
3,5001,1,1,0
4,5002,1,1,0
5,5003,1,1,0
6,5004,1,1,0
7,5005,1,1,0
8,5006,1,1,0


In [7]:
def evaluate_genre(genre): 
    
    providers = ["protected"]
    proItems_genre_n =  unfair_rankingdf.loc[unfair_rankingdf[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    #rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(unfair_rankingdf, unfair_rankingdf, genre, providers)
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [8]:
testproviders = ["protected"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)   

protected
done protected


In [9]:
output1 = pd.concat(allResults, keys=testproviders)
output1

,,rND,rKL,rRD,APCR,nd_APCR,ndcg
,Algorithm,,,,,,
protected,1,0.674279,0.340051,0.809139,1.0,1.0,1.0
